### Configuration

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import mne

import utils__config

In [2]:
# You will need to install the Kleen Lab's IED detector manually by downloading it from GitHub,
# dragging it into your environment library folder, and renaming it. Then drag the contents of
# the linelength_event_detector sub-folder into the main folder. Conda doesn't like importing
# packages (linelength_event_detector) if they have a sub-folder with the exact same name.

from linelength_event_detector.lleventdetector import lleventdetector
from linelength_event_detector.lltransform import lltransform
from linelength_event_detector.lldetect_events import lldetect_events

In [3]:
os.chdir(utils__config.working_directory)
os.getcwd()

'g:\\My Drive\\Residency\\Research\\Lab - Damisah\\Project - Sleep'

### Parameters

In [4]:
fif_path = 'Cache/Subject01/S01_Feb02_256hz.fif'
dictionary_path = 'Data/Subject01/S01_dictionary.xlsx'
bad_channel_path = 'Cache/Subject01/S01_bad_channels.csv'
ied_path = 'Cache/Subject01/S01_IED.csv'

# fif_path = 'Cache/Subject02/Apr26/S02_Apr26_256hz.fif'
# dictionary_path = 'Data/Subject02/S02_dictionary.xlsx'
# bad_channel_path = 'Cache/Subject02/Apr26/S02_bad_channels.csv'
# ied_path = 'Cache/Subject02/Apr26/S02_IED.csv'

# fif_path = 'Cache/Subject02/Apr27/S02_Apr27_256hz.fif'
# dictionary_path = 'Data/Subject02/S02_dictionary.xlsx'
# bad_channel_path = 'Cache/Subject02/Apr27/S02_bad_channels.csv'
# ied_path = 'Cache/Subject02/Apr27/S02_IED.csv'

In [6]:
sampling_freq = 256

### IED Detection

In [7]:
raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = False)

# Select only macroelectrodes
raw.pick_types(seeg = True, ecog = True)

# Remove bad channels
bad_channels = pd.read_csv(bad_channel_path)
bad_channels = bad_channels[bad_channels['channel'].isin(raw.ch_names)]
raw.drop_channels(ch_names = bad_channels['channel'].astype('string'))
print('Channel count after bad channel removal:', len(raw.ch_names))

C:\Users\Layton\AppData\Local\Temp\ipykernel_19800\4201120138.py:1: RuntimeWarning: This filename (Cache/Subject01/S01_Feb02_256hz.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = False)


Channel count after bad channel removal: 68


In [8]:
ied_data = raw.get_data()

#  Inputs: d = list of float values (EEG data)
#          sfx = scalar (sampling frequency)

# Outputs: ets = 2d list (each row is one event; column 0 is start times
#                of each event; column 1 is stop times)
#          ech = 1d list of strings (each string is 1 event, each element
#                in string is the number of the channel involved)

ets, ech = lldetect_events(d = ied_data, sfx = sampling_freq)

In [9]:
# Split the channel numbers by their common divider, a comma
channels = pd.DataFrame(ech)[0].str.split(',')

# Convert IED times into a pandas dataframe for easier munging
ied_times = pd.DataFrame(data=ets, columns=['start', 'stop'])

# Initialize iterators
ied_df = pd.DataFrame()
ied_number = 0

# For every list of channels in the big list
for index in np.arange(0, len(channels)):
    # select the IED start/stop times for this IED
    times = ied_times.iloc[index]

    # advance the IED unique identifier
    ied_number = ied_number + 1

    # for every channel in the list of channels
    # that observed this particular IED
    for sub_index in np.arange(0, len(channels[index])):
        # since each list ends with a comma, each IED
        # will have a fake "empty" channel its end
        if channels[index][sub_index] == '':
            continue

        ch_index = int(channels[index][sub_index])
        ch_name = raw.ch_names[ch_index]

        # create the entry for this channel-IED combo
        ied_series = pd.DataFrame((ied_number,
                                   ch_name,
                                   ied_times.iloc[index][0],
                                   ied_times.iloc[index][1]))

        # add to the growing big dataframe
        ied_df = pd.concat([ied_df, ied_series.transpose()])

# Formatting
ied_df.columns = ['ied_number', 'name', 'start_time', 'stop_time']
ied_df.ied_number = ied_df.ied_number.astype(int)

# Merge with channel meta-data
macro_df = pd.read_excel(dictionary_path)
ied_df = ied_df.merge(macro_df, on='name')
ied_df['ROI'] = ied_df['laterality'] + '_' + ied_df['region']

# Convert start/stop times from 'samples' to 'seconds'
ied_df['start_time'] = ied_df['start_time'] / sampling_freq
ied_df['stop_time'] = ied_df['stop_time'] / sampling_freq

In [10]:
ied_df.to_csv(ied_path, index = False)